# Writing out a USGSCSM ISD from an ISIS ideal camera image

In [1]:
import os
import json

import ale
from ale.drivers.isis_ideal_drivers import IdealLsIsisLabelIsisSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm

## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in an ISIS ideal line scan image using updated, attached exterior orientation data. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `IdealLsIsisLabelIsisSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * An ISIS ideal line scan image (e.g. a deijjtered HiRISE image, an undistorted HRSC image)
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [2]:
# change to desired PDS3 image path
file_name = '/Users/jmapel/ale/ESP_016076_2175_REDmos_hijitreged.balance.noproj.8bit.cub'

from ale.transformation import FrameChain, create_rotations
# metakernels are furnished when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with IdealLsIsisLabelIsisSpiceDriver(file_name) as driver:
    usgscsmString = to_usgscsm(driver)

### Write ISD to disk 

ALE formatter functions generally return bytes or a string that can be written out to disk. ALE's USGSCSM formatter function returns a JSON encoded string that can be written out using any JSON library. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.

In [3]:
# Load the json string into a dict
usgscsm_dict = json.loads(usgscsmString)

# Write the dict out to the associated file
json_file = os.path.splitext(file_name)[0] + '.json'

# Save off the json and read it back in to check if
# the json exists and was formatted correctly
with open(json_file, 'w') as fp:
    json.dump(usgscsm_dict, fp)
    
with open(json_file, 'r') as fp:
    usgscsm_dict = json.load(fp)
    
usgscsm_dict.keys()

dict_keys(['radii', 'sensor_position', 'sun_position', 'sensor_orientation', 'detector_sample_summing', 'detector_line_summing', 'focal_length_model', 'detector_center', 'starting_detector_line', 'starting_detector_sample', 'focal2pixel_lines', 'focal2pixel_samples', 'optical_distortion', 'image_lines', 'image_samples', 'name_platform', 'name_sensor', 'reference_height', 'name_model', 'interpolation_method', 'line_scan_rate', 'starting_ephemeris_time', 'center_ephemeris_time', 't0_ephemeris', 'dt_ephemeris', 't0_quaternion', 'dt_quaternion'])